### Rectify all images on S3 bucket and write to public read bucket

In [ ]:
from pathlib import Path
import imageio
import fsspec
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil import tz
import pandas as pd

from coastcam_funcs import *
from calibration_crs import *
from rectifier_crs import *

#### Read the inventory of products  
Generated with `aws s3 ls s3://cmgp-coastcam/cameras/caco-01/products/ --profile coastcam > caco01inventory.txt`  
Then edited to remove a few bad entries.
These have beeen deleted from the S3 bucket.

In [ ]:
df = pd.read_csv('caco01inventory.txt', header=None, delim_whitespace=True, parse_dates={'datetime': [0, 1]})
df.rename(columns={2:'fid',3:'filename'}, inplace=True)
del df['fid']
df

### Get list of `c1.timex` files and find matching `c2.timex` files. Make list of `c1.timex` images with matches.

In [ ]:
# find instances of timex.c1
df2 = df[df['filename'].str.contains('c1.timex')]
df2

In [ ]:
# filelist will contain all of the c1.timex images that have matching c2 images
filelist = [ ]
matches = 0
no_matches = 0
for i, row in df2.iterrows():
    fs1 = row['filename']
    # print(fs1)
    fn1 = list(fs1)
    fn2 = fn1.copy()
    fn2[12]='2'
    fs2="".join(fn2)
    # print(fs2)
    if len(df[df['filename'].isin([fs2])]) == 1:
        filelist.append(fs1)
        matches += 1
    else:
        no_matches += 1

print(matches, no_matches, matches+no_matches)

### Set up the rectification

In [ ]:
# List of files...three for each camera. Calibration parameters are in .json format
# These are the USGS image filename format
extrinsic_cal_files = ['CACO01_C1_EOBest.json','CACO01_C2_EOBest.json']
intrinsic_cal_files = ['CACO01_C1_IOBest.json','CACO01_C2_IOBest.json']

# Dict providing the metadata that the Axiom code infers from the USACE filename format
metadata= {'name': 'CACO-01', 'serial_number': 1, 'camera_number': 'C1', 'calibration_date': '2019-12-12', 'coordinate_system': 'geo'}
# dict providing origin and orientation of the local grid
local_origin = {'x': 410935.,'y':4655890., 'angd': 55.}

# read cal files and make lists of cal dicts
extrinsics_list = []
for f in extrinsic_cal_files:
    extrinsics_list.append( json2dict(f) )
intrinsics_list = []
for f in intrinsic_cal_files:
    intrinsics_list.append( json2dict(f) )
    
# check test for coordinate system
if metadata['coordinate_system'].lower() == 'xyz':
    print('Extrinsics are local coordinates')
elif metadata['coordinate_system'].lower() == 'geo':
    print('Extrinsics are in world coordinates')
else:
    print('Invalid value of coordinate_system: ',metadata['coordinate_system'])
    
print(extrinsics_list[0])
print(extrinsics_list[0]['y']-local_origin['y'])

calibration = CameraCalibration(metadata,intrinsics_list[0],extrinsics_list[0],local_origin)
print(calibration.local_origin)
print(calibration.world_extrinsics)
print(calibration.local_extrinsics)

xmin = 0.
xmax = 500.
ymin = 0.
ymax = 700.
dx = 1.
dy = 1.
z =  0.

rectifier_grid = TargetGrid(
    [xmin, xmax],
    [ymin, ymax],
    dx,
    dy,
    z
)
#print(rectifier_grid.X)

rectifier = Rectifier(
    rectifier_grid
)

In [ ]:
# setup S3 buckets
imdir='cmgp-coastcam/cameras/caco-01/products/'
fs = fsspec.filesystem('s3',profile='coastcam')

fs2 = fsspec.filesystem('s3', profile='default')


In [ ]:
# function to create a c2 image name from a c1 image name
def c1toc2(c1name):
    fn1 = list(c1name)
    fn2 = fn1.copy()
    fn2[12]='2'
    fs2="".join(fn2)
    return fs2

# test it
print(filelist[22])
print(c1toc2(filelist[22]))

In [ ]:
# loop through the filelist and calculate image brightness and sharpness.
for i, fn in enumerate(filelist):
    fpath = imdir+fn
    ofile = fn.split('.')[0]+'.jpg'
    s = estimate_sharpness(fpath,fs)
    rgb, avg = average_color(fpath,fs)
    ftime, e = filetime2timestr(fn, timezone='eastern')
    impaths = [fpath, imdir+c1toc2(fn)]
#     print(ofile)
#     print(impaths)
#     print(i,fn,ftime,avg,s)
    if avg > 40. :
        rectified_image, _, _, _ = rectifier.rectify_images(metadata, impaths, intrinsics_list, \
                                                            extrinsics_list, local_origin, fs=fs)        
        # write to the public sfm bucket
        with fs2.open('s3://cmgp-sfm-public-read-bucket/csherwood/caco01_rectified/'+ofile,'wb') as fo:
                    imageio.imwrite(fo,np.flip(rectified_image,0),format='jpg')